# solaccel: a recommendation algorithm for residential solar development

In order for the US to reach carbon-neutral by 2035, the growth of renewable energy resources needs to be accelerated dramatically. One important piece of the renewable energy pie is residential rooftop solar, which adds reseliency to the energy grid while powering entire homes with clean, renewable energy.

**Question:** Given a wealth of data in the residential space, can we predict which homes are poised to adopt solar and use this information to launch targeted marketing campaigns that accelerate residential solar adoption.

**Proposal:** Build a recommendation algorithm for residential solar developers that scores potential clients using GIS, socio-economic, and other data as inputs.

**Data Sources:** I choose Denver, CO as a test city for project development, given an existing residential solar presence and some of the best open access aerial imaging data of any US city. 
 * [High-Resolution Aerial Map Data of the Denver Metro Area](https://map.openaerialmap.org/#/-104.96612548828124,39.700847368881,14/square/0231010301231013?_k=040hdj)
 * [Real Property Resedential Characteristics]( https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-real-property-residential-characteristics)

**Analysis:**
 * Build an image classifier to identify rooftops with existing solar installations
     * Inspiration - [Google Project Sunroof](https://sunroof.withgoogle.com/data-explorer/place/ChIJy3mhUO0SK4cRrBtKNfjHaYw/#?overlay=installations)
 * Use homes with solar + additional residential data sources (e.g. median income, property value, sun exposure, proximity of other solar installations) to identify factors which indicate "solar readiness". Apply that model to houses without solar to compute a "solar readiness" score.
 * Visualize the "solar readiness" score as a heatmap to identify areas to target with solar marketing campaigns.

**Outlook:** This tool could be extended to data sets beyond residential, including - small commercial, not-for-profit, etc.

## Data Exploration: Rooftop Solar in Residential Denver Suburbs

In [1]:
from ipyleaflet import *

We select OpenAerialMap data of Denver to study rooftop solar. Denver is pretty unique in providing open aerial imagery data with sufficient resolution to resolve solar panels across most of the city's suburbs. Imagery of other metropolitan areas at this resolution (and higher) can be purchased from private firms to extend the analysis if a business case is found.

### Figure 1: 
An overview of a high-resolution aerial image of Washington Park, a neighborhood of Denver.

In [35]:
m = Map(center=(39.70041812520869, -104.9644303314911), zoom=16, max_zoom = 30)
aerial_image_layer = TileLayer(url ="https://tiles.openaerialmap.org/5bcf6a7591037900144c3aa4/0/5bcf6a7591037900144c3aa7/{z}/{x}/{y}",
                              max_zoom=30, max_native_zoom=30)

control = LayersControl(position='topright')

m.add_layer(aerial_image_layer)
m.add_control(control)

m

Map(center=[39.70041812520869, -104.9644303314911], controls=(ZoomControl(options=['position', 'zoom_in_text',…

We can think of using this small block (or others like it) as a training set or test case for a larger analysis.

Rough statistics of the block, at a glance:
* Block Area ~ 0.5 x 0.5mi
* TIFF Image Size ~ 0.4GB
* Aerial Image Resolution: 3in (maybe 25cm - there's a discrepancy on the website)
* Number of Residential Homes: ~320
* Number of Homes with Rooftop Solar: ~19

If we aim to analyze the entire metropolitan area of Denver (20mi)^2, we can expect a factor of ~1600 in scale. An image-based analysis would comb through ~640GB of TIFF data.

### Example 1a: A house with rooftop solar

In [23]:
m = Map(center=(39.69812998438461, -104.96665928437405), zoom=21, max_zoom = 30)
aerial_image_layer = TileLayer(url ="https://tiles.openaerialmap.org/5bcf6a7591037900144c3aa4/0/5bcf6a7591037900144c3aa7/{z}/{x}/{y}",
                              max_zoom=30, max_native_zoom=30)

control = LayersControl(position='topright')

m.add_layer(aerial_image_layer)
m.add_control(control)

m

Map(center=[39.69812998438461, -104.96665928437405], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Example 1b: A house without rooftop solar

In [27]:
m = Map(center=(39.698683066187705, -104.96673971583425), zoom=21, max_zoom = 30)
aerial_image_layer = TileLayer(url ="https://tiles.openaerialmap.org/5bcf6a7591037900144c3aa4/0/5bcf6a7591037900144c3aa7/{z}/{x}/{y}",
                              max_zoom=30, max_native_zoom=30)

control = LayersControl(position='topright')

m.add_layer(aerial_image_layer)
m.add_control(control)

m

Map(center=[39.698683066187705, -104.96673971583425], controls=(ZoomControl(options=['position', 'zoom_in_text…

Here are a few other potentially relevant GIS datasets for Denver:
* 2018 Building Outlines: https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-building-outlines-2018
* Real Property Residential Characteristics: https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-real-property-residential-characteristics
* Tree Canopy: https://www.denvergov.org/opendata/dataset/tree-canopy-2014